In [9]:
#import
import math
import torch

import numpy as np
import pandas as pd
import torch.nn as nn
import seaborn as sns
import xgboost as xgb
import torch.nn as nn
import torch.optim as optim
import pytorch_lightning as pl
import torch.nn.functional as F
import matplotlib.pyplot as plt

from random import randint
from scipy.optimize import curve_fit
from sklearn.metrics import mean_squared_error
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler

/Users/andrey.golda/.pyenv/versions/3.9.0/lib/python3.9/site-packages/torchvision/io/image.py:11: UserWarning: Failed to load image Python extension: dlopen(/Users/andrey.golda/.pyenv/versions/3.9.0/lib/python3.9/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN2at4_ops19empty_memory_format4callEN3c108ArrayRefIxEENS2_8optionalINS2_10ScalarTypeEEENS5_INS2_6LayoutEEENS5_INS2_6DeviceEEENS5_IbEENS5_INS2_12MemoryFormatEEE
  Referenced from: <179EF805-4C6A-3E6D-B575-3163BED58F4F> /Users/andrey.golda/.pyenv/versions/3.9.0/lib/python3.9/site-packages/torchvision/image.so
  Expected in:     <9D4C7FD2-49A8-383A-AC3E-A560DE81B0D8> /Users/andrey.golda/.pyenv/versions/3.9.0/lib/python3.9/site-packages/torch/lib/libtorch_cpu.dylib
  warn(f"Failed to load image Python extension: {e}")


In [5]:
df = pd.read_csv('./data/clasdb_pi_plus_n.txt', delimiter='\t', header=None)
df.columns = ['Ebeam', 'W', 'Q2', 'cos_theta', 'phi', 'dsigma_dOmega', 'error', 'id']

df.loc[8314:65671, 'Ebeam'] = 5.754
df['phi'] = df.phi.apply(lambda x: math.radians(x))
df = df.iloc[df[['Ebeam', 'W', 'Q2', 'cos_theta', 'phi']].drop_duplicates().index]

In [6]:
# Creating a Data Set
class PhysicsDataSet(Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels
        self.len = len(labels)

    def __getitem__(self, index):
        feature = self.features[index]
        label = self.labels[index]
        return feature, label

    def __len__(self):
        return self.len

In [7]:
feature_data = df[['Ebeam', 'W', 'Q2', 'cos_theta', 'phi']]
label_data = df['dsigma_dOmega']

# Splitting Data into Train and Test

scaler_feature = StandardScaler()
scaler_target = StandardScaler()
feature_data = scaler_feature.fit_transform(feature_data)
label_data = scaler_target.fit_transform(label_data.values.reshape(-1,1))

train_feature_data, test_feature_data, train_label_data, test_label_data = train_test_split(feature_data, label_data, test_size=0.2, random_state=42)


# Creating Data Loaders
train_dataset = PhysicsDataSet(train_feature_data, train_label_data)
test_dataset = PhysicsDataSet(test_feature_data, test_label_data)

train_loader = DataLoader(dataset = train_dataset, batch_size = 8, shuffle = True)
test_loader = DataLoader(dataset = test_dataset, batch_size = 8, shuffle = False)

# FCNN

In [24]:
class CustomNet(nn.Module):
    def __init__(self):
        super().__init__()
        input_dim = 5
        hidden_dim_1 = 60
        hidden_dim_2 = 80
        hidden_dim_3 = 100
        hidden_dim_4 = 120
        hidden_dim_5 = 140
        hidden_dim_6 = 100
        hidden_dim_7 = 80
        hidden_dim_8 = 60
        output_dim = 1
        
        self.net = nn.Sequential(
                    nn.Linear(input_dim, hidden_dim_1),
                    nn.ReLU(),
                    nn.Linear(hidden_dim_1, hidden_dim_2),
                    nn.ReLU(),
                    nn.Linear(hidden_dim_2, hidden_dim_3),
                    nn.ReLU(),
                    nn.Linear(hidden_dim_3, hidden_dim_4),
                    nn.ReLU(),
                    nn.Linear(hidden_dim_4, hidden_dim_5),
                    nn.ReLU(),
                    nn.Linear(hidden_dim_5, hidden_dim_6),
                    nn.ReLU(),
                    nn.Linear(hidden_dim_6, hidden_dim_7),
                    nn.ReLU(),
                    nn.Linear(hidden_dim_7, hidden_dim_8),
                    nn.ReLU(),
                    nn.Linear(hidden_dim_8, output_dim)
                )

    def forward(self, x):
        return self.net(x)

In [25]:
CustomNet()

CustomNet(
  (net): Sequential(
    (0): Linear(in_features=5, out_features=60, bias=True)
    (1): ReLU()
    (2): Linear(in_features=60, out_features=80, bias=True)
    (3): ReLU()
    (4): Linear(in_features=80, out_features=100, bias=True)
    (5): ReLU()
    (6): Linear(in_features=100, out_features=120, bias=True)
    (7): ReLU()
    (8): Linear(in_features=120, out_features=140, bias=True)
    (9): ReLU()
    (10): Linear(in_features=140, out_features=100, bias=True)
    (11): ReLU()
    (12): Linear(in_features=100, out_features=80, bias=True)
    (13): ReLU()
    (14): Linear(in_features=80, out_features=60, bias=True)
    (15): ReLU()
    (16): Linear(in_features=60, out_features=1, bias=True)
  )
)

In [26]:
class CustomModule(pl.LightningModule):
    def __init__(self, model):
        super().__init__()
        self.model = model

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = nn.MSELoss(y_hat, y)
        self.log('train_loss', loss)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = nn.MSELoss(y_hat, y)
        self.log('val_loss', loss)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=0.001)
        return optimizer

In [30]:
model = CustomModule(CustomNet())
trainer = pl.Trainer(max_epochs=100)
trainer.fit(model, train_loader, test_loader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: /Users/andrey.golda/Documents/Study/MSU_interpol/lightning_logs

  | Name  | Type      | Params
------------------------------------
0 | model | CustomNet | 69.5 K
------------------------------------
69.5 K    Trainable params
0         Non-trainable params
69.5 K    Total params
0.278     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/Users/andrey.golda/.pyenv/versions/3.9.0/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:432: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


RuntimeError: mat1 and mat2 must have the same dtype

In [ ]:
class CustomNet(nn.Module):
    def __init__(self):
        super(CustomNet, self).__init__()
        # Define the layers of your network
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(input_dim, hidden_dim_1)
        self.fc2 = nn.Linear(hidden_dim_1, hidden_dim_2)
        self.fc3 = nn.Linear(input_dim_2, hidden_dim_3)
        self.fc4 = nn.Linear(hidden_dim_3, hidden_dim_4)
        self.fc5 = nn.Linear(input_dim_4, hidden_dim_5)
        self.fc6 = nn.Linear(hidden_dim_5, hidden_dim_6)
        self.fc7 = nn.Linear(input_dim_6, hidden_dim_7)
        self.fc8 = nn.Linear(hidden_dim_7, hidden_dim_8)
        self.fc8 = nn.Linear(hidden_dim_7, hidden_dim_8)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x